In [27]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import Adam
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

import sys
sys.path.append("../../")
import pandas as pd
import os
import numpy as np
import DeepStrain.functions_collection as ff
from tqdm.auto import tqdm


# logistic regression

In [30]:
# Define the logistic regression model
class LogisticRegression(nn.Module):
    def __init__(self,
                  input_size, hidden_size = 128):
        super(LogisticRegression, self).__init__()
        self.input_size = input_size
        self.hidden = nn.Linear(input_size, hidden_size)  # Hidden layer
        self.output = nn.Linear(hidden_size, 1)  # Output layer for binary classification

    def forward(self, x):
        x = F.relu(self.hidden(x))  # Sigmoid activation for hidden layer
        out = torch.sigmoid(self.output(x))  # Sigmoid activation for output layer
        return out

class LogisticRegression_Trainer():
    def __init__(
        self,
        regression_model,
        train_num_steps,
        save_folder,

        train_lr = 1e-3,
        train_lr_decay_every = 1,
        save_models_every = 2,):
        

        super().__init__()
        self.model = regression_model
        self.input_size = regression_model.input_size
        self.save_folder = save_folder; os.makedirs(self.save_folder, exist_ok=True)

        # loss:
        self.criterion = nn.BCELoss()

        # optimizer
        self.opt = Adam(self.model.parameters(), lr = train_lr, betas = (0.9, 0.99))
        # self.opt = optim.SGD(model.parameters(), lr=0.001)
        self.scheduler = StepLR(self.opt, step_size = 1, gamma=0.95)
        self.train_lr_decay_every = train_lr_decay_every
        self.step = 0
        self.train_num_steps = train_num_steps
        self.save_models_every = save_models_every

    def simple_val(self,trained_model, x, y):
        y_pred = []; y_pred_float = []

        for index in range(0,x.shape[0]):
            new_data = x[index,:]

            trained_model.eval()

            with torch.no_grad():
                prediction = trained_model(new_data); y_pred_float.append(prediction.item())
                predicted_class = 1 if prediction.item() > 0.5 else 0; y_pred.append(predicted_class)

        y_pred = np.asarray(y_pred); y_pred_float = np.asarray(y_pred_float); y_true = torch.clone(y).numpy()

        # calculate accuracy using predict_collect and ground truth
        accuracy, sensitivity, specificity,_,_,_,_ = ff.quantitative(y_pred, y_true)
        return y_pred, y_pred_float, accuracy, sensitivity, specificity

    def save_model(self, stepNum):
        data = {
            'step': self.step,
            'model': self.model.state_dict(),
            'opt': self.opt.state_dict(),
            'scheduler': self.scheduler.state_dict(),}
        os.makedirs(os.path.join(self.save_folder, 'models'), exist_ok=True)
        torch.save(data, os.path.join(self.save_folder, 'models', 'model-' + str(stepNum) + '.pt'))

    def load_model(self, trained_model_filename):
        data = torch.load(trained_model_filename)

        self.model.load_state_dict(data['model'])

        self.step = data['step']
        self.opt.load_state_dict(data['opt'])
        self.scheduler.load_state_dict(data['scheduler'])


    def train(self, X_train, Y_train, X_val, Y_val, pre_trained_model = None ,start_step = None):
    
        training_log = []

        # load pre-trained
        if pre_trained_model is not None:
            self.load_model(pre_trained_model)
            print('model loaded from ', pre_trained_model)

        if start_step is not None:
            self.step = start_step

        with tqdm(initial = self.step, total = self.train_num_steps) as pbar:
            
            while self.step < self.train_num_steps:
                # print('training epoch: ', self.step + 1)
                # print('learning rate: ', self.scheduler.get_last_lr()[0])

                self.opt.zero_grad()

                # Forward pass
                outputs = self.model(X_train)

                # Calculate loss
                loss = self.criterion(outputs.view(-1), Y_train.float())  # BCELoss expects float inputs

                loss.backward()
                self.opt.step()

                self.step += 1

                # save the model
                if self.step !=0 and (self.step % self.save_models_every == 0):
                   self.save_model(self.step)
                
                if self.step !=0 and (self.step % self.train_lr_decay_every == 0):
                    self.scheduler.step()

                if self.step !=0 and (self.step % self.save_models_every == 0):
                    _,_, train_accuracy, train_sensitivity, train_specificity = self.simple_val(self.model, X_train, Y_train)
                    _,_, val_accuracy, val_sensitivity, val_specificity = self.simple_val(self.model, X_val, Y_val)
                    print('epoch is: ', self.step, ' train loss: ', loss.item(), ' train accuracy: ', train_accuracy, ' sensitivity: ', train_sensitivity, ' specificity: ', train_specificity, ' val accuracy: ', val_accuracy, ' sensitivity: ', val_sensitivity, ' specificity: ', val_specificity) 
                            
                    # save the training log
                    training_log.append([self.step,self.scheduler.get_last_lr()[0], loss.item(), train_accuracy, train_sensitivity, train_specificity, val_accuracy, val_sensitivity, val_specificity])
                    df = pd.DataFrame(training_log,columns = ['iteration','learning_rate', 'train_loss', 'train_accuracy', 'train_sensitivity', 'train_specificity', 'val_accuracy', 'val_sensitivity', 'val_specificity'])
                    log_folder = os.path.join(self.save_folder,'log');ff.make_folder([log_folder])
                    df.to_excel(os.path.join(log_folder, 'training_log.xlsx'),index=False)

                pbar.update(1)




# XGBoost

In [57]:
import torch
import torch.nn as nn
import xgboost as xgb
from sklearn.metrics import accuracy_score

class XGBoostClassifier(nn.Module):
    def __init__(self, input_dim, save_folder, num_classes, n_estimators=100, learning_rate=1e-3, max_depth=3, use_gpu=True):
        super(XGBoostClassifier, self).__init__()
        self.input_dim = input_dim
        self.num_classes = num_classes
        self.save_folder = save_folder
        self.n_estimators = n_estimators
        self.learning_rate = learning_rate
        self.max_depth = max_depth
        self.use_gpu = use_gpu
        
        # Initialize an XGBoost model
        self.xgb_model = xgb.XGBClassifier(
            n_estimators=self.n_estimators,
            learning_rate=self.learning_rate,
            max_depth=self.max_depth,
            objective='multi:softprob',
            num_class=self.num_classes,
            use_label_encoder=False,
            eval_metric='mlogloss',
            tree_method='gpu_hist' if self.use_gpu else 'auto',
            predictor='gpu_predictor' if self.use_gpu else 'auto'
        )


    def forward(self, x):
        x_np = x.detach().cpu().numpy()
        pred_prob = self.xgb_model.predict_proba(x_np)
        pred_prob_tensor = torch.tensor(pred_prob, dtype=torch.float32, device=x.device)
        return pred_prob_tensor

    def fit(self, x_train, y_train, x_val=None, y_val=None):
        x_train_np = x_train.detach().cpu().numpy()
        y_train_np = y_train.detach().cpu().numpy()
        
        if x_val is not None and y_val is not None:
            x_val_np = x_val.detach().cpu().numpy()
            y_val_np = y_val.detach().cpu().numpy()
        
        training_log = []
        for epoch in range(self.n_estimators):
            self.xgb_model.n_estimators = epoch + 1
            self.xgb_model.fit(x_train_np, y_train_np, eval_set=[(x_val_np, y_val_np)] if x_val is not None else None, verbose=False)
            
            # Calculate training accuracy
            train_preds = self.xgb_model.predict(x_train_np)
            train_preds = np.argmax(train_preds, axis=1)
            train_acc = accuracy_score(y_train_np, train_preds)
            
            # Print training accuracy
            print(f"Epoch {epoch + 1}/{self.n_estimators}")
            print(f"Training Accuracy: {train_acc:.4f}")
            
            if x_val is not None:
                # Calculate validation accuracy
                val_preds = self.xgb_model.predict(x_val_np)
                val_preds = np.argmax(val_preds, axis=1)
                val_acc = accuracy_score(y_val_np, val_preds)
                
                # Print validation accuracy
                print(f"Validation Accuracy: {val_acc:.4f}")
            print("-" * 30)

            if epoch % 10 == 0:
                # save the training log
                training_log.append([epoch, train_acc, val_acc])
                df = pd.DataFrame(training_log,columns = ['iteration', 'train_accuracy', 'val_accuracy'])
                log_folder = os.path.join(self.save_folder,'log');ff.make_folder([log_folder])
                df.to_excel(os.path.join(log_folder, 'training_log.xlsx'),index=False)

    def predict(self, x):
        x_np = x.detach().cpu().numpy()
        preds = self.xgb_model.predict(x_np)
        preds_tensor = torch.tensor(preds, dtype=torch.long, device=x.device)
        return preds_tensor


In [12]:
class simple_DataLoader():
    def __init__(self, spreadsheet_path, data_path, index_range):
        
        super().__init__()
        self.spreadsheet = pd.read_excel(spreadsheet_path)
        self.data_path = data_path
        self.index_range = index_range

        self.patient_list = self.spreadsheet.iloc[self.index_range]

    def load_X(self):
        X = []
        for patient_index in range(0,self.patient_list.shape[0]):
            patient_id_num = self.patient_list['OurID'].iloc[patient_index]
            patient_id = ff.XX_to_ID_00XX(patient_id_num)

            # load strain first:
            strain_folder = os.path.join(self.data_path, 'results/strain', patient_id)
            # global strain
            global_strain = np.load(os.path.join(strain_folder, 'ED_ES_by_AI_volume/global_strain.npy'))
            global_radial_strain = np.asarray(global_strain[0]).reshape(1,-1)
            global_circumferential_strain = np.asarray(global_strain[1]).reshape(1,-1)

            # layer strain
            layer_strain = np.load(os.path.join(strain_folder, 'ED_ES_by_AI_volume/layer_strain.npy'))
            layer_radial_strain = layer_strain[0,:].reshape(1,-1)
            layer_circumferential_strain = layer_strain[1,:].reshape(1,-1)

            # polar strain
            polar_strain = np.load(os.path.join(strain_folder, 'ED_ES_by_AI_volume/polar_strain.npy'), allow_pickle=True)
            Ecc_aha = np.asarray(polar_strain[5][:-1]).reshape(1,-1)
            Err_aha = np.asarray(polar_strain[7][:-1]).reshape(1,-1)

            # # WTCI
            # wtci = np.load(os.path.join(strain_folder, 'ED_ES_by_AI_volume/wtci.npy'), allow_pickle=True)
            # wtci_aha = np.asarray(wtci[4][:-1]).reshape(1,-1)

            # load geometry then:
            geometry_folder = os.path.join(self.data_path, 'results/geometry', patient_id)
            # circular index
            circular_index = np.load(os.path.join(geometry_folder, 'circular_index_collect.npy')).reshape(1,-1)
            # centers
            centers = np.load(os.path.join(geometry_folder, 'centers_collect.npy'), allow_pickle=True)
            center_delta_to_base = centers[1][1:,:]
            # euclidean distance from center_delta_to_base, first row is x and second row is y
            center_delta_to_base_euclidean = np.sqrt(np.sum(np.square(center_delta_to_base), axis=1)).reshape(1,-1)
            # enclosed_area = np.asarray(centers[-1]).reshape(1,-1)

            # axis len
            axis = np.load(os.path.join(geometry_folder, 'axis_len_collect.npy'), allow_pickle=True)
            major_axis = axis[:-1,0].reshape(1,-1)
            minor_axis = axis[:-1,1].reshape(1,-1)

            # concatenate all features
            features = np.concatenate((global_radial_strain,   # 0
                                       global_circumferential_strain,  # 1
                                    layer_radial_strain,  #2:5
                                    layer_circumferential_strain, #5:8
                                    Ecc_aha, #8:24
                                    #np.concatenate((Err_aha[:,0:12] , wtci_aha[:,12:16]),axis = 1), #24:40
                                    Err_aha,
                                    # geometry
                                    circular_index, #40:50
                                    center_delta_to_base_euclidean, #50:59
                                    major_axis, #59:69
                                    minor_axis), #69:79
                                    axis=1)
            X.append(features)
        X = np.squeeze(np.asarray(X))
        return torch.from_numpy(X).float()

    def load_Y(self):
        Y = self.spreadsheet['label_for_ML'].iloc[self.index_range]
        return torch.from_numpy(Y.to_numpy()).float()
        

# Main script: 

In [14]:
# load data
spreadsheet_path = '/mnt/camca_NAS/HFpEF/data/HFpEF_data/Patient_list/Important_HFpEF_Patient_list_unique_patient_w_readmission_finalized.xlsx' 

data_path = '/mnt/camca_NAS/Deepstrain'
index_range = range(0,50)
dataloader = simple_DataLoader( spreadsheet_path, data_path, index_range)
X = dataloader.load_X()
Y = dataloader.load_Y()

In [56]:
# main script for logistic regression: 
save_folder = '/mnt/camca_NAS/Deepstrain/HFpEF_analysis/models/logistic_regression/'
X_train, Y_train,  train_idx, X_val, Y_val, val_idx = ff.split_train_val(X, Y, 5, 1 , save_split_file = '/mnt/camca_NAS/Deepstrain/HFpEF_analysis/models/logistic_regression/data_split1_5groups.npy')
print('train idx: ', train_idx, ' val idx: ', val_idx)
model = LogisticRegression(input_size = X_train.shape[-1])
trainer = LogisticRegression_Trainer(model,  train_num_steps = 100000,
                  save_folder = save_folder,
                  train_lr = 1e-3,  train_lr_decay_every = 100,   save_models_every = 100,)

trainer.train(X_train, Y_train, X_val, Y_val, pre_trained_model = None ,start_step = None)


train idx:  [ 1  3  5  6  7  0  2  4  9 11 16 19 21 23 24 25 27 30 33 34 26 28 29 31
 32 37 39 40 41 42 35 36 38 43 45 49 44 46 47 48]  val idx:  [ 8 10 13 14 15 12 17 18 20 22]


  0%|          | 0/100000 [00:00<?, ?it/s]

  0%|          | 188/100000 [00:01<04:55, 338.02it/s]

epoch is:  100  train loss:  0.5182775855064392  train accuracy:  0.75  sensitivity:  0.7368421052631579  specificity:  0.7619047619047619  val accuracy:  0.2  sensitivity:  0.0  specificity:  0.4
epoch is:  200  train loss:  0.320993036031723  train accuracy:  0.9  sensitivity:  0.8947368421052632  specificity:  0.9047619047619048  val accuracy:  0.4  sensitivity:  0.4  specificity:  0.4


  0%|          | 283/100000 [00:01<04:13, 393.14it/s]

...

epoch is:  6500  train loss:  0.0  train accuracy:  1.0  sensitivity:  1.0  specificity:  1.0  val accuracy:  0.5  sensitivity:  0.4  specificity:  0.6


  7%|▋         | 6600/100000 [01:00<13:58, 111.40it/s]

epoch is:  6600  train loss:  0.0  train accuracy:  1.0  sensitivity:  1.0  specificity:  1.0  val accuracy:  0.5  sensitivity:  0.4  specificity:  0.6


  7%|▋         | 6603/100000 [01:00<14:15, 109.13it/s]


KeyboardInterrupt: 

In [58]:
# main script for XGBoost
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
X_train, Y_train,  train_idx, X_val, Y_val, val_idx = ff.split_train_val(X, Y, 5, 1 , save_split_file = '/mnt/camca_NAS/Deepstrain/HFpEF_analysis/models/logistic_regression/data_split1_5groups.npy')

# Initialize the XGBoost classifier
model = XGBoostClassifier(input_dim=X_train.shape[1], num_classes=2, use_gpu=torch.cuda.is_available(), save_folder='/mnt/camca_NAS/Deepstrain/HFpEF_analysis/models/xgboost/', n_estimators=1000)

# Train the model with training and validation data
model.fit(X_train, Y_train.long(), X_val, Y_val.long())

# # Predict on new data
# x_test = torch.randn(20, 10).to(device)
# predictions = model.predict(x_test)

# # Get prediction probabilities
# probabilities = model(x_test)

# print(predictions)
# print(probabilities)


/usr/local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:23:51] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:23:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


Epoch 1/1000
Training Accuracy: 0.9250
Validation Accuracy: 0.2000
------------------------------


/usr/local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:23:51] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:23:51] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:23:52] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:23:52] WARNING: /workspace/sr

Epoch 2/1000
Training Accuracy: 0.9750
Validation Accuracy: 0.3000
------------------------------
Epoch 3/1000
Training Accuracy: 0.9750
Validation Accuracy: 0.5000
------------------------------


/usr/local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:23:52] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:23:52] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:23:52] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:23:52] WARNING: /workspace/sr

...

Epoch 144/1000
Training Accuracy: 1.0000
Validation Accuracy: 0.5000
------------------------------


/usr/local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:25:43] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:25:43] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:25:44] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Epoch 145/1000
Training Accuracy: 1.0000
Validation Accuracy: 0.5000
------------------------------


/usr/local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:25:44] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:25:44] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "predictor", "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)
/usr/local/lib/python3.8/site-packages/xgboost/core.py:158: UserWarning: [16:25:45] WARNING: /workspace/src/common/error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)


Epoch 146/1000
Training Accuracy: 1.0000
Validation Accuracy: 0.5000
------------------------------
